In [1]:
import os
import sys

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from models import new_models
from config import load_data

from ray import tune
from ray.air.integrations.mlflow import MLflowLoggerCallback
from ray.tune.schedulers import ASHAScheduler
from torch.optim.lr_scheduler import ReduceLROnPlateau

import mlflow
from mlflow.tracking import MlflowClient

In [2]:
ld = load_data(data_dir = "./data/",
target_variable = 'Q_Kalltveit')
vars = [None, ["Nedbør Nilsebu"], ["Q_Lyngsaana"], ["Nedbør Nilsebu", "Q_Lyngsaana"]]
X, y = ld.create_lagged_matrix(25, vars_to_lag=["Nedbør Nilsebu", "Q_Lyngsaana"])

In [3]:
def fit(net, loss_function, optimizer, data_loader, num_epochs, mode, lr_scheduler, use_amp=False):
	history = {"train": {"loss": [], "mae": []}, "val": {"loss": [], "mae": []}}
	scaler = torch.cuda.amp.GradScaler(enabled=use_amp) # Mixed-precision support for compatible GPUs
	print("\nTraining the model:")
	for epoch in range(num_epochs):
		print("\nEpoch", epoch+1)
		if epoch < num_epochs - 1:
			keys = ["train", "val"]
		else:
			keys = ["train", "val", "test"]
		for key in keys:
			dataset_size = 0
			dataset_loss = 0.0
			if key == "train":
				net.train()
			else:
				net.eval()
			for X_batch, y_batch in tqdm(data_loader[key]):
				X_batch, y_batch = X_batch.to(mode["device"]), y_batch.to(mode["device"])
				with torch.set_grad_enabled(mode=(key=="train")): # Autograd activated only during training
					with torch.cuda.amp.autocast(enabled=use_amp): # Mixed-precision support for compatible GPUs
						batch_output = net(X_batch.float())
						batch_loss = loss_function(batch_output, y_batch)
					if key == "train":
						scaler.scale(batch_loss).backward()
						scaler.step(optimizer) 	
						scaler.update()
						optimizer.zero_grad()
				dataset_size += y_batch.shape[0]
				dataset_loss += y_batch.shape[0] * batch_loss.item()
			dataset_loss /= dataset_size
			if key in ["train", "val"]:
				history[key]["loss"].append(dataset_loss)
				if key == "train":
					tune.report(train_loss=dataset_loss)
				else:
					val_loss = dataset_loss
					lr_scheduler.step(metrics=val_loss)
					tune.report(val_loss=val_loss)
			else:
				print("\nEvaluating the model:")
				print(key, "loss:", dataset_loss)
				tune.report(test_loss=dataset_loss)
	return net

In [4]:
from config import load_data

def train_model(config, data_dir):

    use_GPU = torch.cuda.is_available()
    if use_GPU:
        mode = {"name": "cuda", "device": torch.device("cuda")}
    else:
        mode = {"name": "cpu", "device": torch.device("cpu")}

    # Define hyperparameters
    train_size = 0.6
    val_size = 0.2
    test_size = 0.2

    sequence_length = config['sequence_length']
    batch_size = config['batch_size']
    num_epochs = config['num_epochs']
    lr = config['lr']
    weight_decay = config['weigth_decay']

    ld = load_data(data_dir = data_dir,
        target_variable = config['target_variable'])
    
    vars = ["Nedbør Nilsebu", "Q_Lyngsaana"]
    pca = config['pca']
    mi = config['mi']
    
    X, y = ld.create_lagged_matrix(window_size=sequence_length)

    X_train, y_train, X_val, y_val, X_test, y_test = ld.split_data(X, y)


    train_dataloader = ld.create_dataloader(X_train, y_train, sequence_length, batch_size=batch_size, shuffle=True)
    val_dataloader = ld.create_dataloader(X_val, y_val, sequence_length, batch_size=batch_size, shuffle=True)
    test_dataloader = ld.create_dataloader(X_test, y_test, sequence_length, batch_size=batch_size, shuffle=False)
    
    # Model inputs
    input_size = X_train.shape[-1]
    hidden_size = config['hidden_size']
    num_layers = config['num_layers']
    output_size = 1

    if config['arch'] == "FCN":
        net = new_models.FCN(input_size,
                    hidden_size,
                    num_layers,
                    output_size,
                    )
    elif config['arch'] ==  "FCNTemporalAttention":
        net = new_models.FCNTemporalAttention(input_size,
                    hidden_size,
                    num_layers,
                    output_size,
                    )
    elif config['arch'] == "LSTM":
        net = new_models.LSTM(input_size,
                    hidden_size,
                    num_layers,
                    output_size,
                    )
    elif config['arch'] == "LSTMTemporalAttention":
        net = new_models.LSTMTemporalAttention(input_size,
                    hidden_size,
                    num_layers,
                    output_size,
                    )

    data_loader = {
    "train": train_dataloader,
    "val": val_dataloader,
    "test": test_dataloader,
    }
    
    net.to(mode["device"])

    loss_function = nn.MSELoss().to(mode["device"])
    optimizer = optim.Adam(net.parameters(), lr=lr, weight_decay=weight_decay)

    # Define your learning rate scheduler
    lr_scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5, verbose=True)
                                           
    scaler = torch.cuda.amp.GradScaler(enabled=False) # Mixed-precision support for compatible GPUs
    for epoch in range(num_epochs):
        print("\nEpoch", epoch+1)
        if epoch < num_epochs - 1:
            keys = ["train", "val"]
        else:
            keys = ["train", "val", "test"]
        for key in keys:
            dataset_size = 0
            dataset_loss = 0.0
            if key == "train":
                net.train()
            else:
                net.eval()
            for X_batch, y_batch in tqdm(data_loader[key]):
                X_batch, y_batch = X_batch.to(mode["device"]), y_batch.to(mode["device"])
                with torch.set_grad_enabled(mode=(key=="train")): # Autograd activated only during training
                    with torch.cuda.amp.autocast(enabled=False): # Mixed-precision support for compatible GPUs
                        batch_output = net(X_batch.float())
                        batch_loss = loss_function(batch_output, y_batch)
                    if key == "train":
                        scaler.scale(batch_loss).backward()
                        scaler.step(optimizer) 	
                        scaler.update()
                        optimizer.zero_grad()
                dataset_size += y_batch.shape[0]
                dataset_loss += y_batch.shape[0] * batch_loss.item()

            dataset_loss /= dataset_size

            # Report results to Ray Tune
            if key == "train":
                tune.report(train_loss=dataset_loss)
            elif key == "val":
                # Update learning rate
                lr_scheduler.step(metrics=dataset_loss)
                tune.report(val_loss=dataset_loss)
            else:
                print("\nEvaluating the model:")
                print(key, "loss:", dataset_loss)
                tune.report(test_loss=dataset_loss)

                
    out_name = ""
    for k, v in config.items():
        if not k in ['weights_dir', 'cwd']:
            out_name += '{}-{}_'.format(k, v)
    torch.save(net.state_dict(), os.path.join(config['cwd'], config['weights_dir'], out_name[:-1] + '.pth'))

In [5]:
client = MlflowClient()
cwd = os.getcwd()
exp_base_name = "Test_of_interface"

created = 0
for i in range(100):
    try:
        exp_name = exp_base_name+"_{}".format(i)
        experiment_id = client.create_experiment(exp_name)
        created=1
        break
    except (TypeError, mlflow.exceptions.MlflowException):
        continue

if not created:
    print("ERROR: Try new experiment name.")
    sys.exit(1)

weights_root = "./model_weights/"
weights_dir = weights_root+exp_name+'/'
os.mkdir(weights_dir)

In [6]:
data_dir = "./data/"
target_variable = 'Q_Kalltveit'

In [7]:
from functools import partial

config = {
    "mlflow_experiment_id": experiment_id,
    "weights_dir": weights_dir,
    "cwd": cwd,
    "target_variable": target_variable,
    "arch": tune.grid_search(["FCN", "FCNTemporalAttention", "LSTM", "LSTMTemporalAttention"]), # , "LSTMTemporalAttention"
    #'variables': tune.grid_search(), # , ["Q_Lyngsaana"], ["Nedbør Nilsebu", "Q_Lyngsaana"]
    "sequence_length": tune.grid_search([25, 40, 50, 60, 70]),
    'num_epochs': tune.grid_search([150]),
    'num_layers': tune.choice([2, 3, 4]),
    "lr": tune.loguniform(1e-4, 1e-1),
    "weigth_decay": tune.choice([0, 0.001, 0.0001]),
    "batch_size": tune.choice([128, 256]),
    "hidden_size": tune.grid_search([64]),
    #"pca": tune.choice([1, 0]),
    #"mi": tune.choice([1, 0]),
}


analysis = tune.run(
    partial(train_model, data_dir=data_dir),
    config=config,
    resources_per_trial={"cpu": 12, "gpu": 1},
    num_samples=2,
    callbacks=[MLflowLoggerCallback(experiment_name=exp_name)],
)

Trial name,date,experiment_id,hostname,node_ip,pid,timestamp,trial_id
train_model_5f077_00000,2023-02-23_09-38-00,d6701bf5b6234968ae770a74c4d37b08,DESKTOP-D4IVECG,127.0.0.1,3192,1677141480,5f077_00000
train_model_5f077_00001,2023-02-23_09-38-04,9a5e29957a324ad7a69709016ad3a3e1,DESKTOP-D4IVECG,127.0.0.1,20816,1677141484,5f077_00001
train_model_5f077_00002,2023-02-23_09-38-08,0b751acecf0c4c0a8ae9de13b29dde59,DESKTOP-D4IVECG,127.0.0.1,23116,1677141488,5f077_00002
train_model_5f077_00003,2023-02-23_09-38-12,8faddd9cac1f44c5a63d2a4318b706ea,DESKTOP-D4IVECG,127.0.0.1,12792,1677141492,5f077_00003
train_model_5f077_00004,2023-02-23_09-38-16,d26fb523984e4e479a5fe873970042e3,DESKTOP-D4IVECG,127.0.0.1,26916,1677141496,5f077_00004
train_model_5f077_00005,2023-02-23_09-38-20,5b4d036bacb9426d828ce60f5ea4438e,DESKTOP-D4IVECG,127.0.0.1,19836,1677141500,5f077_00005
train_model_5f077_00006,2023-02-23_09-38-24,7017e038e33346cd8fbf67672f3b8161,DESKTOP-D4IVECG,127.0.0.1,16724,1677141504,5f077_00006
train_model_5f077_00007,2023-02-23_09-38-28,3c6e7a43b27c44fc8575717abcd62daa,DESKTOP-D4IVECG,127.0.0.1,26656,1677141508,5f077_00007
train_model_5f077_00008,2023-02-23_09-38-32,3ba7587af8764ed589464f18d918df0e,DESKTOP-D4IVECG,127.0.0.1,26708,1677141512,5f077_00008
train_model_5f077_00009,2023-02-23_09-38-36,eebcc854a57c46f79e690c96d3c03111,DESKTOP-D4IVECG,127.0.0.1,14072,1677141516,5f077_00009
